# API «Яндекс-Директ» - Базовые функции управления рекламной кампанией

In [1]:
# https://github.com/esliklientovnet/YaDirect_lib/blob/master/__init__.py
# https://github.com/esliklientovnet/sync_minus/blob/master/sync_minus.py#L120    

In [1]:
import pandas as pd
# import yaDirect
import time
import gspread
from oauth2client.service_account import ServiceAccountCredentials

# Вывод всех операций в Jupyter
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

import logging
# Чтобы выводить на экран
logger = logging.getLogger()
logger.setLevel(logging.INFO)


from lib.yaDirect import *

### Параметры доступа к Я-Д

In [2]:
login = '***************'
token = '*****************************************'

In [3]:
# Установить соединения с API Яндекс-Директ
direct = yaDirect(token, login )

In [3]:
def result_to_df(result, FieldNames):
    """
    Преобразуем result (в виде списка словарей) в Датафрейм
    """
    # Получаем ДАННЫЕ
    data_list = []
    for el in result:
        current_date = []
        for column in FieldNames:
            current_date.append(el[column])
        data_list.append(current_date)

    # Делаем датафрейм
    df = pd.DataFrame(data_list, columns=FieldNames)
    return df

# Campaigns

### get - Возвращает список РК

In [5]:
# https://yandex.ru/dev/direct/doc/ref-v5/campaigns/get.html
# Список итоговых полей в таблице
FieldNames = ["Id", "Name"]

# Создание тела запроса
params = {}
params["SelectionCriteria"] = {}
params["FieldNames"] = FieldNames

service = 'campaigns'
method = 'get'
result = direct.requestAPI(service, method, params, login).json()
result

# Вывод df
# result = result['result']['Campaigns']
# try:
#     df = result_to_df(result, FieldNames)
#     df.head(8)
# except:
#     print("Данных нет")

{'result': {'Campaigns': [{'Name': 'Rus_БУ смартфоны-svyaznoy',
    'Id': 65628948},
   {'Name': 'MSK_Поиск_БУ', 'Id': 65628984},
   {'Name': 'SPb_Поиск_БУ', 'Id': 65628995},
   {'Id': 65629004, 'Name': 'Поиск|Общие'},
   {'Name': 'ВЧ_Поиск_конверсии', 'Id': 65629014},
   {'Id': 66053066, 'Name': 'Смарт-баннеры'},
   {'Id': 66054403, 'Name': 'Динамические объявления'},
   {'Id': 66089423, 'Name': 'Rus_РСЯ_БУ'},
   {'Id': 66089435, 'Name': 'Ретаргетинг'},
   {'Id': 66089444, 'Name': 'РСЯ|БУ'},
   {'Id': 66089460, 'Name': 'SPb_РСЯ_БУ'},
   {'Name': 'РСЯ|Общие', 'Id': 66089478},
   {'Name': 'ВЧ|РСЯ|Конверсии', 'Id': 66089484},
   {'Name': 'Поиск|Бренд', 'Id': 66418007},
   {'Name': 'Конкуренты', 'Id': 66419231},
   {'Name': 'Автотаргетинг_Поиск', 'Id': 70625765},
   {'Id': 71310926, 'Name': 'Поиск_доп.ассортимент'},
   {'Name': 'Поиск|Samsung (Клики)', 'Id': 72383433},
   {'Name': 'Поиск|Samsung', 'Id': 72383443},
   {'Id': 72383448, 'Name': 'Поиск|Apple (Биддер)'},
   {'Id': 72383458, 'N

### add - Создание РК с типом "TextCampaign"

**ВНИМАНИЕ!!!**
Существуют отдельные (разные) структуры запроса по типам РК:
* TextCampaign
* SmartCampaign
* DynamicTextCampaign
* MobileAppCampaign
* CpmBannerCampaign

In [6]:
# https://yandex.ru/dev/direct/doc/ref-v5/campaigns/add.html#input__CampaignAddItem
# https://yandex.ru/dev/direct/doc/ref-v5/campaigns/add-text-campaign.html#add-text-campaign__TextCampaignAddItem

# Создание тела запроса
# Параметры типа РК (текстовая РК)
TypeParameters = {}
TypeParameters['BiddingStrategy'] = {}
TypeParameters['BiddingStrategy']['Search'] = {}
# HIGHEST_POSITION - «Ручное управление ставками с оптимизацией»
TypeParameters['BiddingStrategy']['Search']['BiddingStrategyType'] = 'HIGHEST_POSITION'
# TypeParameters['BiddingStrategy']['Search']['WbMaximumClicks'] = {}
# TypeParameters['BiddingStrategy']['Search']['WbMaximumClicks']['WeeklySpendLimit'] = 400000000 
TypeParameters['BiddingStrategy']['Network'] = {}
# MAXIMUM_COVERAGE Ручное управление ставками с оптимизацией», включено раздельное управление ставками на поиске и в сетях
TypeParameters['BiddingStrategy']['Network']['BiddingStrategyType'] = 'MAXIMUM_COVERAGE'
# TypeParameters['BiddingStrategy']['Network']['NetworkDefault'] = {}

# Параметры РК
Campaign = {}
Campaign['Name'] = "Test campaigns 4"
Campaign['StartDate'] = "2024-01-01"
# Параметры типа РК (текстовая РК)
Campaign['TextCampaign'] = TypeParameters

params = {}
params['Campaigns'] = [Campaign]

service = 'campaigns'
method = "add"
result = direct.requestAPI(service, method, params, login).json()
result

{'result': {'AddResults': [{'Id': 76475566}]}}

### delete - Удаление РК

In [11]:
# Удаление РК
# https://yandex.ru/dev/direct/doc/ref-v5/campaigns/delete.html

# Создание тела запроса
params = {}
params['SelectionCriteria'] = {}
params['SelectionCriteria']['Ids'] = [76205878]

service = 'campaigns'
method = "delete"
result = direct.requestAPI(service, method, params, login).json()
result

{'result': {'DeleteResults': [{'Id': 76205878}]}}

### get - Получить параметры РК (ОБЩИЕ + по конкретному типу РК)

In [40]:
# https://yandex.ru/dev/direct/doc/ref-v5/campaigns/get.html

# id РК
Campaigns_id = 76037266

# Создание тела запроса
params = {}
params["SelectionCriteria"] = {}
params["SelectionCriteria"]['Ids'] = [Campaigns_id]

# Список итоговых полей в таблице
FieldNames = ["Id", "Name", "StartDate", "Type", "BlockedIps", 'Status', 'State', 'StatusPayment', 'StatusClarification', 'NegativeKeywords']
params["FieldNames"] = FieldNames

# Список итоговых полей в таблице - по КОНКРЕТНОМУ ТИПУ РК
TextCampaignFieldNames = [ "CounterIds", "RelevantKeywords", "Settings", "AttributionModel", "BiddingStrategy" ]
params["TextCampaignFieldNames"] = TextCampaignFieldNames

service = 'campaigns'
method = "get"
result = direct.requestAPI(service, method, params, login).json()
result

{'result': {'Campaigns': [{'Type': 'TEXT_CAMPAIGN',
    'Id': 76037266,
    'TextCampaign': {'BiddingStrategy': {'Network': {'BiddingStrategyType': 'MAXIMUM_COVERAGE'},
      'Search': {'BiddingStrategyType': 'HIGHEST_POSITION'}},
     'RelevantKeywords': None,
     'CounterIds': None,
     'AttributionModel': 'LYDCCD',
     'Settings': [{'Option': 'ADD_TO_FAVORITES', 'Value': 'NO'},
      {'Option': 'REQUIRE_SERVICING', 'Value': 'NO'},
      {'Option': 'SHARED_ACCOUNT_ENABLED', 'Value': 'YES'},
      {'Value': 'YES', 'Option': 'DAILY_BUDGET_ALLOWED'},
      {'Value': 'NO', 'Option': 'MAINTAIN_NETWORK_CPC'},
      {'Value': 'NO', 'Option': 'ENABLE_SITE_MONITORING'},
      {'Value': 'YES', 'Option': 'ADD_METRICA_TAG'},
      {'Value': 'NO', 'Option': 'ADD_OPENSTAT_TAG'},
      {'Value': 'YES', 'Option': 'ENABLE_EXTENDED_AD_TITLE'},
      {'Value': 'YES', 'Option': 'ENABLE_COMPANY_INFO'},
      {'Value': 'NO', 'Option': 'CAMPAIGN_EXACT_PHRASE_MATCHING_ENABLED'},
      {'Value': 'NO', 'Op

### get - получить МИНУС-ФРАЗЫ (из кампании)

In [53]:
# https://yandex.ru/dev/direct/doc/ref-v5/campaigns/get.html
# Список итоговых полей в таблице
FieldNames = ["Id", "Name", "NegativeKeywords"]

# Создание тела запроса
params = {}
params["SelectionCriteria"] = {}
params["SelectionCriteria"]["Ids"] = [76475566] 
params["FieldNames"] = FieldNames

service = 'campaigns'
method = 'get'
result = direct.requestAPI(service, method, params, login).json()
result

{'result': {'Campaigns': [{'Name': 'Test campaigns 4',
    'Id': 76475566,
    'NegativeKeywords': {'Items': ['восемь', 'пять шесть']}}]}}

### update - Изменить параметры 1 РК (Меняю стратегию РК)

In [302]:
# https://yandex.ru/dev/direct/doc/dg/best-practice/modify.html
# https://yandex.ru/dev/direct/doc/ref-v5/campaigns/update-text-campaign.html
    
# id РК
Campaigns_id = 76037266

# Параметры типа РК (текстовая РК)
TypeParameters = {}
TypeParameters['BiddingStrategy'] = {}
TypeParameters['BiddingStrategy']['Search'] = {}
TypeParameters['BiddingStrategy']['Search']['BiddingStrategyType'] = 'HIGHEST_POSITION'

TypeParameters['BiddingStrategy']['Network'] = {}
TypeParameters['BiddingStrategy']['Network']['BiddingStrategyType'] = 'MAXIMUM_COVERAGE'

# Параметры РК
Campaign = {}
Campaign['Id'] = Campaigns_id
Campaign['TextCampaign'] = TypeParameters

params = {}
params['Campaigns'] = [Campaign]

service = 'campaigns'
method = "update"
result = direct.requestAPI(service, method, params, login).json()
result

{'result': {'UpdateResults': [{'Id': 76037266}]}}

### update - изменить МИНУС-ФРАЗЫ (РК) 

In [52]:
# https://yandex.ru/dev/direct/doc/dg/best-practice/modify.html
# https://yandex.ru/dev/direct/doc/ref-v5/campaigns/update-text-campaign.html
    
# id РК
Campaigns_id = 76475566

# Параметры РК
Campaign = {}
Campaign['Id'] = Campaigns_id
Campaign['NegativeKeywords'] = {}
Campaign['NegativeKeywords']['Items'] = [ 'пять шесть', 'восемь']

params = {}
params['Campaigns'] = [Campaign]


service = 'campaigns'
method = "update"
result = direct.requestAPI(service, method, params, login).json()
result

{'result': {'UpdateResults': [{'Id': 76475566}]}}

# AdGroups - Группа объявлений

### add - Создание группы объявлений

In [18]:
# https://yandex.ru/dev/direct/doc/ref-v5/adgroups/adgroups.html
# https://yandex.ru/dev/direct/doc/ref-v5/adgroups/add.html

# id РК
Campaigns_id = 76475566

# Массив идентификаторов регионов, для которых показы включены или выключены. Массив должен содержать хотя бы один элемент.
# Идентификатор 0 — показывать во всех регионах.
RegionIds = [0]

# Параметры AdGroup (группы объявлений)
AdGroup = {}
AdGroup['Name'] = "Тестовая группа объявлений - 01"
AdGroup['CampaignId'] = Campaigns_id
AdGroup['RegionIds'] = RegionIds

params = {}
params['AdGroups'] = [AdGroup]

service = 'adgroups'
method = "add"
result = direct.requestAPI(service, method, params, login).json()
result

{'result': {'AddResults': [{'Id': 4973667406}]}}

### get - Получить параметры группы объявлений

In [54]:
# https://yandex.ru/dev/direct/doc/ref-v5/adgroups/adgroups.html
    # https://yandex.ru/dev/direct/doc/ref-v5/adgroups/get.html

# Можно указать id-РК, тогда получим все группы

# id РК
Campaign_id = 76475566
# id AdGroup
# AdGroup_id = 4962656425

# Список итоговых полей в таблице
FieldNames = ["Id", "Name", "NegativeKeywords", "NegativeKeywordSharedSetIds"]

# Создание тела запроса
params = {}
params["SelectionCriteria"] = {}
params["SelectionCriteria"]['CampaignIds'] = [Campaign_id]
# params["SelectionCriteria"]['Ids'] = [AdGroup_id]
params["FieldNames"] = FieldNames

service = 'adgroups'
method = "get"
result = direct.requestAPI(service, method, params, login).json()
result

{'result': {'AdGroups': [{'Id': 4973667406,
    'Name': 'Тестовая группа объявлений - 01',
    'NegativeKeywords': None,
    'NegativeKeywordSharedSetIds': {'Items': [37694479]}}]}}

### delete - Удалить группу объявлений

In [51]:
# https://yandex.ru/dev/direct/doc/ref-v5/adgroups/adgroups.html
# https://yandex.ru/dev/direct/doc/ref-v5/adgroups/delete.html

# Создание тела запроса
params = {}
params["SelectionCriteria"] = {}
params["SelectionCriteria"]['Ids'] = [4966393372]

service = 'adgroups'
method = "delete"
result = direct.requestAPI(service, method, params, login).json()
result

{'result': {'DeleteResults': [{'Id': 4966393372}]}}

### update - Изменяет параметры групп объявлений - подключаем набор МИНУС-ФРАЗ

In [21]:
# https://yandex.ru/dev/direct/doc/ref-v5/adgroups/adgroups.html
# https://yandex.ru/dev/direct/doc/ref-v5/adgroups/update.html

# id AdGroup
AdGroup_id = 4973667406
NegativeKeywordSharedSet_Id = 37694479


# Параметры AdGroup (группы объявлений)
AdGroup = {}
AdGroup['Id'] = AdGroup_id
AdGroup['NegativeKeywordSharedSetIds'] = {}
AdGroup['NegativeKeywordSharedSetIds']["Items"] = [NegativeKeywordSharedSet_Id]

params = {}
params['AdGroups'] = [AdGroup]

service = 'adgroups'
method = "update"
result = direct.requestAPI(service, method, params, login).json()
result

{'result': {'UpdateResults': [{'Id': 4973667406}]}}

# Ads - объявления

### add - Создание объявления

In [24]:
# https://yandex.ru/dev/direct/doc/ref-v5/ads/ads.html
# https://yandex.ru/dev/direct/doc/ref-v5/ads/add.html

# id AdGroup
AdGroup_id = 4973667406


# Параметры Ad (объявления)
Ad = {}
Ad['AdGroupId'] = AdGroup_id
Ad['TextAd'] = {}
Ad['TextAd']['Title'] = "Заголовок тестового объявления"
Ad['TextAd']['Text'] = "Основной текст тестового объявления"
Ad['TextAd']['Mobile'] = "NO"
Ad['TextAd']['Href'] = "https://test.ru"

params = {}
params['Ads'] = [Ad]

service = 'ads'
method = "add"
result = direct.requestAPI(service, method, params, login).json()
result

{'result': {'AddResults': [{'Id': 12443945420}]}}

### get - Получить параметры объявления

In [26]:
# https://yandex.ru/dev/direct/doc/ref-v5/ads/ads.html
# https://yandex.ru/dev/direct/doc/ref-v5/adgroups/get.html

# Можно указать id-РК, тогда получим ВСЕ объявления
# Или указать id группы или id - конкретного объявления

# id РК
Campaign_id = 76475566
# id AdGroup
AdGroup_id = 4962656425

# Список итоговых полей в таблице
FieldNames = ["Id", "AdCategories", 'CampaignId', 'AdGroupId']

# Создание тела запроса
params = {}
params["SelectionCriteria"] = {}
params["SelectionCriteria"]['CampaignIds'] = [Campaign_id]

# Для получения параметров ВЕРХНЕГО уровня
params["FieldNames"] = FieldNames

# Для получения КОНКРЕТНЫХ параметров ТЕКСТОВО_ГРАФИЧЕСКОГО объявления
params["TextAdFieldNames"] = ['Title', 'Text', 'Mobile', 'Href']

service = 'ads'
method = "get"
result = direct.requestAPI(service, method, params, login).json()
result

{'result': {'Ads': [{'Id': 12443945420,
    'CampaignId': 76475566,
    'AdGroupId': 4973667406,
    'AdCategories': None,
    'TextAd': {'Text': 'Основной текст тестового объявления',
     'Title': 'Заголовок тестового объявления',
     'Href': 'https://test.ru',
     'Mobile': 'NO'}}]}}

# Keywords - выполнение операций с ключевыми фразами и автотаргетингами

### add - Создание ключевые фразы

In [56]:
# https://yandex.ru/dev/direct/doc/ref-v5/keywords/keywords.html
# https://yandex.ru/dev/direct/doc/ref-v5/keywords/add.html

# Ключевые фразы прикрепляет к группе объявлений

# Можно указывать ставку (Если в кампании настроена ручная стратегия, то можно указать параметр Bid)

# id AdGroup
AdGroup_id = 4973667406

Keyword_1 = {}
Keyword_1['Keyword'] = "Вторая ключевая фраза"
Keyword_1['AdGroupId'] = AdGroup_id
# Ставка на поиске (не обязательно)
Keyword_1['Bid'] = 1000000
# Ставка в сетях (не обязательно)
Keyword_1['ContextBid'] = 2000000

params = {}
params['Keywords'] = [Keyword_1]

service = 'keywords'
method = "add"
result = direct.requestAPI(service, method, params, login).json()
result

{'result': {'AddResults': [{'Id': 39998321635}]}}

### get - Возвращает параметры ключевых фраз

In [54]:
# https://yandex.ru/dev/direct/doc/ref-v5/keywords/keywords.html
# https://yandex.ru/dev/direct/doc/ref-v5/keywords/get.html

# Можно указать id: РК, группы, объявления (?), фраз

# id РК
Campaign_id = 76475566

# Список итоговых полей в таблице
FieldNames = ['Id', 'CampaignId', 'AdGroupId', "Keyword", 'Bid']

# Создание тела запроса
params = {}
params["SelectionCriteria"] = {}
params["SelectionCriteria"]['CampaignIds'] = [Campaign_id]

# Для получения параметров ВЕРХНЕГО уровня
params["FieldNames"] = FieldNames

# # Для получения КОНКРЕТНЫХ параметров ТЕКСТОВО_ГРАФИЧЕСКОГО объявления
# params["TextAdFieldNames"] = ['Title', 'Text', 'Mobile', 'Href']

service = 'keywords'
method = "get"

result = direct.requestAPI(service, method, params, login).json()
result

{'result': {}}

### keywords-update - изменение ключевые фразы

Синтаксис ТОЧНО такой же - как add, только вместо AdGroupId - Id (фразы)

In [31]:
# https://yandex.ru/dev/direct/doc/ref-v5/keywords/keywords.html
# https://yandex.ru/dev/direct/doc/ref-v5/keywords/update.html

# id AdGroup
Id_1 = 40046155627

Keyword_1 = {}
Keyword_1['Id'] = Id_1

Keyword_1['Keyword'] = "Измененная ключевая фраза"

params = {}
params['Keywords'] = [Keyword_1]

service = 'keywords'
method = "update"
result = direct.requestAPI(service, method, params, login).json()
result

{'result': {'UpdateResults': [{'Errors': [{'Code': 8800,
      'Message': 'Объект не найден',
      'Details': 'Ключевое слово не найдено'}]}]}}

In [52]:
# Вытащить список id-Ключевых фраз (для удаления)
my_list_of_dic = result['result']['Keywords']
my_list = []
for el in my_list_of_dic:
    my_list.append(el['Id'])
my_list    

[40184515644, 40184515645]

### keywords-delete - УДАЛЕНИЕ ключевой фразы

In [53]:
# https://yandex.ru/dev/direct/doc/ref-v5/keywords/keywords.html
# https://yandex.ru/dev/direct/doc/ref-v5/keywords/delete.html

Ids = my_list

params = {}
params['SelectionCriteria'] = {}
params['SelectionCriteria']['Ids'] = Ids


service = 'keywords'
method = "delete"
result = direct.requestAPI(service, method, params, login).json()
result

{'result': {'DeleteResults': [{'Id': 40184515644}, {'Id': 40184515645}]}}

# KeywordsResearch - предварительная обработка ключевых слов

### deduplicate - предварительная обработка массива ключевых фраз

Метод выполняет следующие операции с ключевыми фразами:
* **Склейка дублей**

Объединение нескольких фраз, отличающихся только минус-словами, формой слов, наличием стоп-слов. Объединенная фраза будет содержать минус-слова всех фраз. Например, фразы заказ окон -жалюзи и окна на заказ -дешево будут заменены фразой окна на заказ -дешево -жалюзи. В частности, при склейке удаляются дубликаты фраз.

* **Кросс-минусовка**

Добавление минус-слова в случае, когда фраза содержится в другой фразе, отличающейся на одно слово. Например, в паре фраз шуба и шуба норковая к первой фразе будет добавлено минус-слово и получится фраза шуба -норковая. В результате поисковому запросу норковая шуба будет соответствовать только одна из фраз, а не обе.

Метод принимает на вход массив, который может содержать:

* новые ключевые фразы без идентификатора;
* существующие ключевые фразы, имеющие идентификатор в Директе или внешней базе данных.

В результате склейки дублей и кросс-минусовки метод возвращает инструкции, какие фразы нужно добавить, изменить или удалить. Формат ответа максимально приближен к формату запроса для методов add, update и delete сервиса Keywords.

In [58]:
# https://yandex.ru/dev/direct/doc/ref-v5/keywordsresearch/keywordsresearch.html
# https://yandex.ru/dev/direct/doc/ref-v5/keywordsresearch/deduplicate.html#deduplicate

# Перечень ключевых фраз
Keyword_1 = {}
Keyword_1['Keyword'] = "заказ окон -жалюзи"
Keyword_2 = {}
Keyword_2['Keyword'] = "окна на заказ -дешево"

# Создание тела запроса
params = {}
params["Keywords"] = [Keyword_1, Keyword_2]
# MERGE_DUPLICATES (склейка дублей) | ELIMINATE_OVERLAPPING (кросс-минусовка)- Если параметр не задан, будут выполнены обе операции 
params["Operation"] = ["MERGE_DUPLICATES"]

service = 'keywordsresearch'
method = "deduplicate"
result = direct.requestAPI(service, method, params, login).json()
result

{'result': {'Add': [{'Keyword': 'окна на заказ -дешево -жалюзи'}]}}

### hasSearchVolume - Для заданных ключевых фраз и регионов формирует предварительный прогноз НАЛИЧИЯ показов по этим фразам в разбивке по типам устройств. Используется при подборе ключевых фраз

**Ограничения**

В течение 60-секундного интервала можно отправить не более 20 запросов для одного рекламодателя.

In [16]:
# https://yandex.ru/dev/direct/doc/ref-v5/keywordsresearch/keywordsresearch.html
# https://yandex.ru/dev/direct/doc/ref-v5/keywordsresearch/hasSearchVolume.html

# Перечень ключевых фраз
Keyword_1 = "строительство деревянных домов"
Keyword_2 = "продажа билетов на Луну"

# Создание тела запроса
params = {}
params['SelectionCriteria'] = {}
params['SelectionCriteria']["Keywords"] = [Keyword_1, Keyword_2]
# Идентификаторы регионов, для которых показы включены или выключены.
# Идентификатор 0 — показывать во всех регионах.
params['SelectionCriteria']["RegionIds"] = [0]

# Список выводимых поей
params["FieldNames"] = ["Keyword", "RegionIds", "AllDevices", "MobilePhones", "Tablets", "Desktops"]

service = 'keywordsresearch'
method = "hasSearchVolume"
result = direct.requestAPI(service, method, params, login).json()
result

{'result': {'HasSearchVolumeResults': [{'Keyword': 'строительство деревянных домов',
    'RegionIds': [0],
    'AllDevices': 'YES',
    'MobilePhones': 'YES',
    'Tablets': 'YES',
    'Desktops': 'YES'},
   {'Keyword': 'продажа билетов на Луну',
    'RegionIds': [0],
    'AllDevices': 'NO',
    'MobilePhones': 'NO',
    'Tablets': 'NO',
    'Desktops': 'NO'}]}}

# NegativeKeywordSharedSets - операции с наборами минус-фраз

https://yandex.ru/dev/direct/doc/dg/objects/negative-keywords.html

Минус-фразы можно настроить несколькими способами. Все способы можно использовать одновременно.

* Перечислить минус-фразы на уровне группы объявлений в параметре NegativeKeywords и на уровне кампании в параметре NegativeKeywords.
* Сформировать наборы минус-фраз с помощью сервиса NegativeKeywordSharedSets. Подключить наборы минус-фраз на уровне группы объявлений, указав идентификаторы наборов в параметре NegativeKeywordSharedSetIds.
* Указать отдельные минус-слова (то есть минус-фразы из одного слова) непосредственно в ключевой фразе в параметре Keyword.

Перенос минус-слов на уровень группы и кампании упрощает настройку и снижает риск превысить ограничение на длину ключевой фразы (4096 символов).

Если минус-фраза, заданная на уровне группы или кампании, полностью входит в состав ключевой фразы, то к данной ключевой фразе минус-фраза не применяется.

### add - Создает наборы минус-фраз

In [330]:
# https://yandex.ru/dev/direct/doc/ref-v5/negativekeywordsharedsets/negativekeywordsharedsets.html
# https://yandex.ru/dev/direct/doc/ref-v5/negativekeywordsharedsets/add.html

# Ограничения:
# Одному рекламодателю может принадлежать не более 30 наборов минус-фраз
# Не более 30 наборов минус-фраз в одном вызове метода

NegativeKeywordSharedSet_1 = {}
NegativeKeywordSharedSet_1['Name'] = "Первый набор МИНУС-ФРАЗ"
NegativeKeywordSharedSet_1['NegativeKeywords'] = ['первая минус', 'вторая минус', 'третья минус']

params = {}
params['NegativeKeywordSharedSets'] = [NegativeKeywordSharedSet_1]

service = 'negativekeywordsharedsets'
method = "add"
result = direct.requestAPI(service, method, params, login).json()
result

{'result': {'AddResults': [{'Id': 37694479}]}}

### get - Возвращает наборы минус-фраз

In [29]:
# https://yandex.ru/dev/direct/doc/ref-v5/negativekeywordsharedsets/negativekeywordsharedsets.html
# https://yandex.ru/dev/direct/doc/ref-v5/negativekeywordsharedsets/get.html


# Создание тела запроса
params = {}
params["SelectionCriteria"] = {}
params["SelectionCriteria"]['Ids'] = [37694479]

# Для получения параметров ВЕРХНЕГО [уровня
params["FieldNames"] = [ "Id" , "Name" , "NegativeKeywords"]


service = 'negativekeywordsharedsets'
method = "get"
result = direct.requestAPI(service, method, params, login).json()
result

{'result': {'NegativeKeywordSharedSets': [{'Id': 37694479,
    'Name': 'Первый набор МИНУС-ФРАЗ',
    'NegativeKeywords': ['вторая минус', 'первая минус', 'третья минус']}]}}

# KeywordBids - управление ставками

### get - Возвращает ставки и приоритеты для ключевых фраз и автотаргетингов

Возвращает ставки и приоритеты для ключевых фраз и автотаргетингов, отвечающих заданным критериям, а также данные, полезные при подборе ставок: данные аукциона по позициям показа на поиске и ставки для охвата различных долей аудитории в сетях.

In [60]:
# https://yandex.ru/dev/direct/doc/ref-v5/keywordbids/keywordbids.html
# https://yandex.ru/dev/direct/doc/ref-v5/keywordbids/get.html

# *** Можно указать ОДИН из ТРЕХ параметров или ВСЕ
Campaign_id = 76475566
# AdGroupIds 
# KeywordIds
# ***

# Список итоговых полей в таблице - ОБЩИЕ параметры
FieldNames = ['CampaignId', 'AdGroupId',  'KeywordId']

# Список итоговых полей - параметри при ПОИСКЕ
SearchFieldNames = ['Bid', 'AuctionBids']

# Список итоговых полей в таблице - параметры в СЕТЯХ
NetworkFieldNames = ['Bid', 'Coverage']

# Создание тела запроса
params = {}
params["SelectionCriteria"] = {}
params["SelectionCriteria"]['CampaignIds'] = [Campaign_id]

# Для получения параметров 
params["FieldNames"] = FieldNames
params["SearchFieldNames"] = SearchFieldNames
params["NetworkFieldNames"] = NetworkFieldNames

service = 'keywordbids'
method = "get"
result = direct.requestAPI(service, method, params, login).json()
result

{'result': {'KeywordBids': [{'Search': {'Bid': 33000000,
     'AuctionBids': {'AuctionBidItems': [{'TrafficVolume': 139,
        'Bid': 2601700000,
        'Price': 428700000},
       {'TrafficVolume': 120, 'Bid': 1610600000, 'Price': 188700000},
       {'TrafficVolume': 119, 'Bid': 1595100000, 'Price': 176700000},
       {'TrafficVolume': 118, 'Bid': 1579600000, 'Price': 164700000},
       {'TrafficVolume': 117, 'Bid': 1533200000, 'Price': 152600000},
       {'TrafficVolume': 116, 'Bid': 302300000, 'Price': 140700000},
       {'TrafficVolume': 115, 'Bid': 298500000, 'Price': 139300000},
       {'TrafficVolume': 114, 'Bid': 289700000, 'Price': 137900000},
       {'TrafficVolume': 113, 'Bid': 286900000, 'Price': 136500000},
       {'TrafficVolume': 112, 'Bid': 284100000, 'Price': 135200000},
       {'TrafficVolume': 111, 'Bid': 281300000, 'Price': 133900000},
       {'TrafficVolume': 110, 'Bid': 277100000, 'Price': 132500000},
       {'TrafficVolume': 109, 'Bid': 276100000, 'Price': 131

### set - Назначает фиксированные ставки и приоритеты для ключевых фраз и автотаргетингов

In [59]:
# https://yandex.ru/dev/direct/doc/ref-v5/keywordbids/keywordbids.html
# https://yandex.ru/dev/direct/doc/ref-v5/keywordbids/set.html

# Ставку или приоритет можно назначить:
# - для отдельной ключевой фразы или автотаргетинга;
# - для всех ключевых фраз и автотаргетинга в группе объявлений;
# - для всех ключевых фраз и автотаргетингов в кампании.

# id РК
Campaign_id = 76475566

KeywordBid_1 = {}
KeywordBid_1['CampaignId'] = Campaign_id
# Ставка на поиске
KeywordBid_1['SearchBid'] = 33000000
# Ставка в сетях
KeywordBid_1['NetworkBid'] = 99000000

# Создание тела запроса
params = {}
params["KeywordBids"] = [KeywordBid_1]

service = 'keywordbids'
method = "set"
result = direct.requestAPI(service, method, params, login).json()
result

{'result': {'SetResults': [{'CampaignId': 76475566}]}}

# BidModifiers - управление корректировками ставок

**Внимание!!!**
Можно подключать к корректировке ставок - "RetargetingLists", т.е устанавливать коррекции для сегментов из Я-Аудиторий !!! 
https://yandex.ru/support/audience/audience-in-direct.html

### add - cоздает корректировки ставок

In [63]:
# https://yandex.ru/dev/direct/doc/ref-v5/bidmodifiers/bidmodifiers.html
# https://yandex.ru/dev/direct/doc/ref-v5/bidmodifiers/add.html

# Соответствие типов корректировок типам кампаний и групп:
# https://yandex.ru/dev/direct/doc/dg/objects/bidmodifier.html#compatibility

# ----- Корректировка - 1 (для мобильных)
BidModifier_1 = {}
# Корректировка на мобильных
BidModifier_1['MobileAdjustment'] = {}
# Значение коэффициента к ставке - в % (от 0 до 1300)
BidModifier_1['MobileAdjustment']['BidModifier'] = 120

# Либо CampaignId, либо AdGroupId
BidModifier_1['CampaignId'] = 76475566
# BidModifier_1['AdGroupId'] = 


# ----- Корректировка - 2 (демография)
DemographicsAdjustment_01 = {}
DemographicsAdjustment_01['Gender'] = "GENDER_MALE"
# Значение коэффициента к ставке - в % (от 0 до 1300)
DemographicsAdjustment_01['BidModifier'] = 170


BidModifier_2 = {}
BidModifier_2['DemographicsAdjustments'] = [DemographicsAdjustment_01]


# Либо CampaignId, либо AdGroupId
BidModifier_2['CampaignId'] = 76475566
# BidModifier_1['AdGroupId'] = 


# Создание тела запроса
params = {}
params["BidModifiers"] = [BidModifier_2]

service = 'bidmodifiers'
method = "add"
result = direct.requestAPI(service, method, params, login).json()
result

{'result': {'AddResults': [{'Ids': [11947093810]}]}}

### get - возвращает параметры корректировок, отвечающих заданным критериям

In [64]:
# https://yandex.ru/dev/direct/doc/ref-v5/bidmodifiers/bidmodifiers.html
# https://yandex.ru/dev/direct/doc/ref-v5/bidmodifiers/get.html

# Создание тела запроса
params = {}
params["SelectionCriteria"] = {}

# Хотя бы один из параметров CampaignIds, AdGroupIds и Ids
params["SelectionCriteria"]['CampaignIds'] = [76475566]
# params["SelectionCriteria"]['AdGroupIds'] = []
# params["SelectionCriteria"]['Ids'] = []

params["SelectionCriteria"]['Levels'] = ["CAMPAIGN", "AD_GROUP"]

# Список выводимых полей - высокий уровень
params["FieldNames"] =[ "Id", "CampaignId", "AdGroupId", "Level", "Type" ]

# Список выводимых полей - по мобильным
params["MobileAdjustmentFieldNames"] = [ "BidModifier", "OperatingSystemType" ]

service = 'bidmodifiers'
method = "get"
result = direct.requestAPI(service, method, params, login).json()
result

{'result': {'BidModifiers': [{'CampaignId': 76475566,
    'AdGroupId': None,
    'Id': 11947093810,
    'Level': 'CAMPAIGN',
    'Type': 'DEMOGRAPHICS_ADJUSTMENT'}]}}

### set - Изменяет значения коэффициентов в корректировках ставок (по сути - update)

In [24]:
# https://yandex.ru/dev/direct/doc/ref-v5/bidmodifiers/bidmodifiers.html
# https://yandex.ru/dev/direct/doc/ref-v5/bidmodifiers/set.html

BidModifier_1 = {}
# Идентификатор корректировки
BidModifier_1['Id'] = 10945233426
# Значение коэффициента к ставке - в % (от 0 до 1300)
BidModifier_1['BidModifier'] = 150

# Создание тела запроса
params = {}
params["BidModifiers"] = [BidModifier_1]

service = 'bidmodifiers'
method = "set"
result = direct.requestAPI(service, method, params, login).json()
result

{'result': {'SetResults': [{'Id': 10945233426}]}}

### toggle - Включает/выключает набор корректировок

In [34]:
# https://yandex.ru/dev/direct/doc/ref-v5/bidmodifiers/bidmodifiers.html
# https://yandex.ru/dev/direct/doc/ref-v5/bidmodifiers/toggle.html

BidModifierToggleItem_1 = {}

# Либо CampaignId, либо AdGroupId
BidModifierToggleItem_1['CampaignId'] = 76037266
# BidModifierToggleItem_1['AdGroupId'] = 

# Тип корректировок, которые нужно включить или выключить: 
# DEMOGRAPHICS_ADJUSTMENT
# RETARGETING_ADJUSTMENT
# REGIONAL_ADJUSTMENT
# SERP_LAYOUT_ADJUSTMENT
# INCOME_GRADE_ADJUSTMENT
BidModifierToggleItem_1['Type'] = 'DEMOGRAPHICS_ADJUSTMENT'

# YES — включить набор корректировок, NO — выключить набор корректировок.
BidModifierToggleItem_1['Enabled'] = "NO"


# Создание тела запроса
params = {}
params["BidModifierToggleItems"] = [BidModifierToggleItem_1]

service = 'bidmodifiers'
method = "toggle"
result = direct.requestAPI(service, method, params, login).json()
result

{'result': {'ToggleResults': [{'CampaignId': 76037266,
    'Type': 'DEMOGRAPHICS_ADJUSTMENT'}]}}

### delete - Удаляет корректировки ставок

In [38]:
# https://yandex.ru/dev/direct/doc/ref-v5/bidmodifiers/bidmodifiers.html
# https://yandex.ru/dev/direct/doc/ref-v5/bidmodifiers/delete.html

# Создание тела запроса
params = {}
params["SelectionCriteria"] = {}
# Идентификаторы корректировок, которые требуется удалить
params["SelectionCriteria"]['Ids'] = [11945245039]

service = 'bidmodifiers'
method = "delete"
result = direct.requestAPI(service, method, params, login).json()
result

{'result': {'DeleteResults': [{'Id': 11945245039}]}}

# RetargetingLists - создание ЛИСТА подбора аудитории - ретаргетинг

### Создает условия ретаргетинга и подбора аудитории (add)

Возвращает ставки и приоритеты для ключевых фраз и автотаргетингов, отвечающих заданным критериям, а также данные, полезные при подборе ставок: данные аукциона по позициям показа на поиске и ставки для охвата различных долей аудитории в сетях.

Для настройки условия ретаргетинга и подбора аудитории используются:
* **Цели и сегменты**, настроенные в **Яндекс-Метрике**
* **Сегменты**, настроенные в **Яндекс-Аудиториях**
* Сегменты на основе данных Крипты: сегменты по социально-демографическим характеристикам и поведенческим признакам, а также сегменты по интересам пользователя, см. раздел Профиль пользователей помощи Директа.

### add - Создаем RetargetingLists (используем цель из Яндекс-Метрики) 

In [66]:
# https://yandex.ru/dev/direct/doc/ref-v5/retargetinglists/retargetinglists.html
# https://yandex.ru/dev/direct/doc/ref-v5/retargetinglists/add.html


RetargetingLists_1 = {}
RetargetingLists_1['Type'] = 'RETARGETING'
RetargetingLists_1['Name'] = 'test_METRIKA_01'

# Правило 1
Argument_1 = {}
# Количество дней (от 1 до 540), за которое проверяется выполнение цели.
Argument_1['MembershipLifeSpan'] = 300
# Идентификатор цели (из Яндекс-Метрики - добавить в корзину) или сегмента
Argument_1['ExternalId'] = 222601045 
# 222601045


# Набор правил 1
Rule_1 = {}
Rule_1['Arguments'] = [Argument_1]
Rule_1["Operator"] = "ALL"

RetargetingLists_1['Rules'] = [Rule_1]


# Создание тела запроса
params = {}
params["RetargetingLists"] = [RetargetingLists_1]


service = 'retargetinglists'
method = "add"
result = direct.requestAPI(service, method, params, login).json()
result

{'result': {'AddResults': [{'Id': 11439536}]}}

### add - Создаем RetargetingLists (используем СЕГМЕНТ из Яндекс-Метрики) 

In [69]:
# https://yandex.ru/dev/direct/doc/ref-v5/retargetinglists/retargetinglists.html
# https://yandex.ru/dev/direct/doc/ref-v5/retargetinglists/add.html


RetargetingLists_1 = {}
RetargetingLists_1['Type'] = 'RETARGETING'
RetargetingLists_1['Name'] = 'test_METRIKA_SEGMENT_01'

# Правило 1
Argument_1 = {}
# Количество дней (от 1 до 540), за которое проверяется выполнение цели.
Argument_1['MembershipLifeSpan'] = 300
# Идентификатор СЕГМЕНТА из Я-Метрики 
Argument_1['ExternalId'] = 1006408065 
# 222601045


# Набор правил 1
Rule_1 = {}
Rule_1['Arguments'] = [Argument_1]
Rule_1["Operator"] = "ALL"

RetargetingLists_1['Rules'] = [Rule_1]


# Создание тела запроса
params = {}
params["RetargetingLists"] = [RetargetingLists_1]


service = 'retargetinglists'
method = "add"
result = direct.requestAPI(service, method, params, login).json()
result

{'result': {'AddResults': [{'Id': 11439593}]}}

### add - Создаем RetargetingLists (сегмент из Яндекс-Аудитории) 

### **Внимание!!!** Перед id (для Яндекс-Аудиторий) необходимо добавлять префикс "20" 

In [71]:
# https://yandex.ru/dev/direct/doc/ref-v5/retargetinglists/retargetinglists.html
# https://yandex.ru/dev/direct/doc/ref-v5/retargetinglists/add.html


RetargetingLists_2 = {}
RetargetingLists_2['Type'] = 'AUDIENCE'
RetargetingLists_2['Name'] = 'test_test_test_02'

# Правило 1
Argument_2 = {}
# Количество дней (от 1 до 540), за которое проверяется выполнение цели - в Я-А - не используется
Argument_2['MembershipLifeSpan'] = 300
# Идентификатор сегмента
# Внимание!!! Перед id (для Яндекс-Аудиторий) необходимо добавлять префикс "20"
Argument_2['ExternalId'] = 2024733284


# Набор правил 1
Rule_2 = {}
Rule_2['Arguments'] = [Argument_2]
Rule_2["Operator"] = "ALL"

RetargetingLists_2['Rules'] = [Rule_2]


# Создание тела запроса
params = {}
params["RetargetingLists"] = [RetargetingLists_2]


service = 'retargetinglists'
method = "add"
result = direct.requestAPI(service, method, params, login).json()
result

{'result': {'AddResults': [{'Id': 11439619}]}}

### get - Возвращает условия ретаргетинга и подбора аудитории

In [72]:
# https://yandex.ru/dev/direct/doc/ref-v5/retargetinglists/retargetinglists.html
# https://yandex.ru/dev/direct/doc/ref-v5/retargetinglists/get.html

params = {}
params["SelectionCriteria"] = {}
params["SelectionCriteria"]['Types'] = [ 'RETARGETING', 'AUDIENCE' ]

# Для получения параметров ВЕРХНЕГО уровня
params["FieldNames"] = ["Type", "Id", "Name", "Description", "Rules"]

service = 'retargetinglists'
method = "get"
result = direct.requestAPI(service, method, params, login).json()
result

{'result': {'RetargetingLists': [{'Type': 'RETARGETING',
    'Id': 8377543,
    'Name': 'Положил в корзину, но не заказал',
    'Description': None,
    'Rules': [{'Arguments': [{'MembershipLifeSpan': 60,
        'ExternalId': 222601045}],
      'Operator': 'ALL'},
     {'Arguments': [{'MembershipLifeSpan': 60, 'ExternalId': 222601153},
       {'MembershipLifeSpan': 60, 'ExternalId': 222601576}],
      'Operator': 'NONE'}]},
   {'Type': 'RETARGETING',
    'Id': 8377566,
    'Name': 'Посетил, но не оформил',
    'Description': None,
    'Rules': [{'Arguments': [{'MembershipLifeSpan': 60,
        'ExternalId': 3065071099}],
      'Operator': 'NONE'},
     {'Arguments': [{'MembershipLifeSpan': 60, 'ExternalId': 4065071099}],
      'Operator': 'ANY'}]},
   {'Type': 'RETARGETING',
    'Id': 9389777,
    'Name': 'просмотр более 3-х стр',
    'Description': None,
    'Rules': [{'Arguments': [{'MembershipLifeSpan': 14,
        'ExternalId': 135311017}],
      'Operator': 'ANY'}]},
   {'Type': 

### delete - Удаляет условия ретаргетинга и подбора аудитории

In [49]:
# https://yandex.ru/dev/direct/doc/ref-v5/retargetinglists/retargetinglists.html
# https://yandex.ru/dev/direct/doc/ref-v5/retargetinglists/delete.html

params = {}
params["SelectionCriteria"] = {}
params["SelectionCriteria"]['Ids'] = [11435573]

service = 'retargetinglists'
method = "delete"
result = direct.requestAPI(service, method, params, login).json()
result

{'result': {'DeleteResults': [{'Id': 11435573}]}}

# AudienceTargets - связывает: RetargetingList + AdGroup

### add - Создаем связь (RetargetingList - AdGroup)

Плюс для условий нацеливания:
* назначаем ставки
* приоритеты 

In [41]:
# https://yandex.ru/dev/direct/doc/ref-v5/audiencetargets/audiencetargets.html
# https://yandex.ru/dev/direct/doc/ref-v5/audiencetargets/add.html

AdGroupId = 4973667406
RetargetingListId = 11153503

AudienceTarget_1 = {}
AudienceTarget_1['AdGroupId'] = AdGroupId
AudienceTarget_1['RetargetingListId'] = RetargetingListId
# размер ставки
AudienceTarget_1['ContextBid'] = 9000000

# Создание тела запроса
params = {}
params["AudienceTargets"] = [AudienceTarget_1]


service = 'audiencetargets'
method = "add"
result = direct.requestAPI(service, method, params, login).json()
result

{'result': {'AddResults': [{'Id': 34108684}]}}

### get - Возвращает связь (RetargetingList - AdGroup)

In [42]:
# https://yandex.ru/dev/direct/doc/ref-v5/audiencetargets/audiencetargets.html
# https://yandex.ru/dev/direct/doc/ref-v5/audiencetargets/get.html

params = {}
params["SelectionCriteria"] = {}
params["SelectionCriteria"]['CampaignIds'] = [ 76475566 ]

# Для получения параметров ВЕРХНЕГО уровня
params["FieldNames"] = [ "Id", "AdGroupId", "CampaignId", "RetargetingListId", "InterestId", "ContextBid", "State"]

service = 'audiencetargets'
method = "get"
result = direct.requestAPI(service, method, params, login).json()
result

{'result': {'AudienceTargets': [{'Id': 34108684,
    'AdGroupId': 4973667406,
    'CampaignId': 76475566,
    'RetargetingListId': 11153503,
    'InterestId': None,
    'State': 'ON',
    'ContextBid': 9000000}]}}

### setBids - Назначает ставки и приоритеты (RetargetingList - AdGroup)

Ставку или приоритет можно назначить для: 
* отдельного условия нацеливания (указываем id-связи) на аудиторию, 
* всех условий нацеливания на аудиторию в группе объявлений (указываем id-группы объявлений) 
* всех условий нацеливания на аудиторию в кампании (указываем id-Кампании).

In [12]:
# https://yandex.ru/dev/direct/doc/ref-v5/audiencetargets/audiencetargets.html
# https://yandex.ru/dev/direct/doc/ref-v5/audiencetargets/setBids.html

Bid_1 = {}

# Либо Id, либо AdGroupId, либо CampaignId
Bid_1['Id'] = 33869868
# Bid_1['AdGroupId'] = 
# Bid_1['CampaignId'] = 
Bid_1['ContextBid'] = 11000000

params = {}
params["Bids"] = [Bid_1]

service = 'audiencetargets'
method = "setBids"
result = direct.requestAPI(service, method, params, login).json()
result

{'result': {'SetBidsResults': [{'Id': 33869868}]}}

### suspend - Остановить показы (RetargetingList - AdGroup) 

После выполнения функции: параметр 'State': 'SUSPENDED'

In [14]:
# https://yandex.ru/dev/direct/doc/ref-v5/audiencetargets/audiencetargets.html
# https://yandex.ru/dev/direct/doc/ref-v5/audiencetargets/suspend.html

params = {}
params["SelectionCriteria"] = {}
params["SelectionCriteria"]["Ids"] = [33869868] 

service = 'audiencetargets'
method = "suspend"
result = direct.requestAPI(service, method, params, login).json()
result

{'result': {'SuspendResults': [{'Id': 33869868}]}}

### resume - Возобновляет  показы (RetargetingList - AdGroup)

После выполнения функции: параметр 'State': 'ON'

In [16]:
# https://yandex.ru/dev/direct/doc/ref-v5/audiencetargets/audiencetargets.html
# https://yandex.ru/dev/direct/doc/ref-v5/audiencetargets/suspend.html

params = {}
params["SelectionCriteria"] = {}
params["SelectionCriteria"]["Ids"] = [33869868] 

service = 'audiencetargets'
method = "resume"
result = direct.requestAPI(service, method, params, login).json()
result

{'result': {'ResumeResults': [{'Id': 33869868}]}}

# Feeds - операций с фидами

### add - Feed - создание фида

In [48]:
# https://yandex.ru/dev/direct/doc/ref-v5/feeds/feeds.html
# https://yandex.ru/dev/direct/doc/ref-v5/feeds/add.html

# Фид можно создавать на основе файла (строки) или на основе страницы на сайте

# ??? - почему то требует указывать и файл и строку с данными в виде base64

# Ограничения:
# - Не более 50 фидов у одного рекламодателя. (1 фид - 1 файл)
# - Не более 50 фидов в одном вызове метода.

# Требования к фиду:
# https://yandex.ru/support/direct/feeds/about.html#feeds__requirements
# https://yandex.ru/support/direct/feeds/requirements.html

# http://base64.ru/

Feed_01 = {}
Feed_01['Name'] = "Feed_1"
Feed_01['BusinessType'] = "RETAIL"
Feed_01['SourceType'] = "FILE"
Feed_01['FileFeed'] = {}
Feed_01['FileFeed']['Data'] = "PG9mZmVyIGlkPSIwMTIzNDU2IiBhdmFpbGFibGU9InRydWUiPgogIDx1cmw+aHR0cDovL3d3dy7QvdCw0LTQtdC20L3QsNGP0YLQtdGF0L3QuNC60LAu0YDRhC9jYXRhbG9nL2VsZW1lbnQvaW5kZXgucGhwP2Zyb209eWFfbWFya2V0JmFtcDt1dG1fc291cmNlPXlhX21hcmtldCZhbXA7dXRtX21lZGl1bT1jcGMmYW1wPC91cmw+CiAgPHByaWNlPjE2MjAuMDA8L3ByaWNlPgogIDxvbGRwcmljZT4xODAwLjAwPC9vbGRwcmljZT4KICA8Y3VycmVuY3lJZD5SVUI8L2N1cnJlbmN5SWQ+CiAgPGNhdGVnb3J5SWQ+MTk8L2NhdGVnb3J5SWQ+CiAgPHBpY3R1cmU+aHR0cDovLzg5LjEyMy40NS42NzgvY2F0YWxvZy9waG90by8xOS82LkpQRzwvcGljdHVyZT4KICA8cGljdHVyZT5odHRwOi8vODkuMTIzLjQ1LjY3OC9jYXRhbG9nL3Bob3RvLzE5LzcuSlBHPC9waWN0dXJlPgogIDxzdG9yZT5mYWxzZTwvc3RvcmU+CiAgPHBpY2t1cD50cnVlPC9waWNrdXA+CiAgPGRlbGl2ZXJ5PmZhbHNlPC9kZWxpdmVyeT4KICA8bmFtZT7QkNC90YLQuNCy0LjRgNGD0YEgRVNFVCBOT0QzMiBQbGF0aW51bSBFZGl0aW9uPC9uYW1lPgogIDx2ZW5kb3I+RXNldDwvdmVuZG9yPgogIDx2ZW5kb3JDb2RlPk5PRDMyLUVOQS1OUyhCT1gpLTItMTwvdmVuZG9yQ29kZT4KICA8ZGVzY3JpcHRpb24+0JDQvdGC0LjQstC40YDRg9GBIEVTRVQgTk9EMzIgUGxhdGludW0gRWRpdGlvbiAtINC70LjRhtC10L3Qt9C40Y8g0L3QsCAyINCz0L7QtNCwIE5PRDMyLUVOQS1OUyhCT1gpLTItMTwvZGVzY3JpcHRpb24+CiAgPHNhbGVzX25vdGVzPtCe0L/Qu9Cw0YLQsDog0J3QsNC70LjRh9C90YvQtSwg0JEv0J0sINC/0LvQsNGB0YLQuNC60L7QstGL0LUg0LrQsNGA0YLRiywg0LrRgNC10LTQuNGCPC9zYWxlc19ub3Rlcz4KICA8bWFudWZhY3R1cmVyX3dhcnJhbnR5PnRydWU8L21hbnVmYWN0dXJlcl93YXJyYW50eT4KICA8Y291bnRyeV9vZl9vcmlnaW4+0KDQvtGB0YHQuNGPPC9jb3VudHJ5X29mX29yaWdpbj4KICA8YWdlIHVuaXQ9InllYXIiPjE4PC9hZ2U+Cjwvb2ZmZXI+"
Feed_01['FileFeed']['Filename'] = "my_feed.yml"

params = {}
params['Feeds'] = [Feed_01]

service = 'feeds'
method = "add"
result = direct.requestAPI(service, method, params, login).json()
result

{'result': {'AddResults': [{'Id': 432892}]}}

### get - Возвращает параметры ФИДА

In [54]:
# https://yandex.ru/dev/direct/doc/ref-v5/feeds/feeds.html
# https://yandex.ru/dev/direct/doc/ref-v5/feeds/get.html


# Создание тела запроса
params = {}
# Отбираем ВСЕ фиды
# params["SelectionCriteria"] = {}
# params["SelectionCriteria"]['Ids'] = [432892]

# Для получения параметров 
params["FieldNames"] = [ "Id" , "Name" , "BusinessType", "Status"]

# Имя исходного файла
params["FileFeedFieldNames"] = [ "Filename" ]

service = 'feeds'
method = "get"
result = direct.requestAPI(service, method, params, login).json()
result

{'result': {'Feeds': [{'Id': 238509,
    'Name': 'Фид 12.10.2021',
    'BusinessType': 'RETAIL',
    'FileFeed': {},
    'Status': 'DONE'},
   {'Id': 432888,
    'Name': 'Feed_1',
    'BusinessType': 'RETAIL',
    'FileFeed': {'Filename': 'my_feed.yml'},
    'Status': 'ERROR'},
   {'Id': 432892,
    'Name': 'Feed_1',
    'BusinessType': 'RETAIL',
    'FileFeed': {'Filename': 'my_feed.yml'},
    'Status': 'ERROR'}]}}

### add - Создание AdGroup c фидом

In [66]:
# https://yandex.ru/dev/direct/doc/ref-v5/adgroups/adgroups.html
# https://yandex.ru/dev/direct/doc/ref-v5/adgroups/add.html

# id РК
Campaigns_id = 76037266

# Массив идентификаторов регионов, для которых показы включены или выключены. Массив должен содержать хотя бы один элемент.
# Идентификатор 0 — показывать во всех регионах.
RegionIds = [0]

# Параметры AdGroup (группы объявлений)
AdGroup = {}
AdGroup['Name'] = "Тестовая - 02 (с фидами)"
AdGroup['CampaignId'] = Campaigns_id
AdGroup['RegionIds'] = RegionIds

# Параметры подключаемого фида
TextAdGroupFeedParams_1 = {}
TextAdGroupFeedParams_1['FeedId'] = 432888
AdGroup['TextAdGroupFeedParams'] = TextAdGroupFeedParams_1

params = {}
params['AdGroups'] = [AdGroup]

service = 'adgroups'
method = "add"
result = direct.requestAPI(service, method, params, login).json()
result

{'result': {'AddResults': [{'Id': 4965209059,
    'Warnings': [{'Code': 10165,
      'Message': 'Parameter will not be applied',
      'Details': 'Товарные дополнения на поиске больше не поддерживаются. Фиды не будут добавлены к группе объявлений.'}]}]}}

# DynamicFeedAdTargets - выполнения операций с условиями нацеливания для динамических объявлений по фидам

### add - Создает условия нацеливания для динамических объявлений, назначает ставки или приоритеты для создаваемых условий 

In [66]:
# https://yandex.ru/dev/direct/doc/ref-v5/dynamicfeedadtargets/dynamicfeedadtargets.html
# https://yandex.ru/dev/direct/doc/ref-v5/dynamicfeedadtargets/add.html

# Параметры AdGroup (группы объявлений)
AdGroup = {}
AdGroup['Name'] = "Тестовая - 02 (с фидами)"
AdGroup['CampaignId'] = Campaigns_id
AdGroup['RegionIds'] = RegionIds

# Параметры подключаемого фида
TextAdGroupFeedParams_1 = {}
TextAdGroupFeedParams_1['FeedId'] = 432888
AdGroup['TextAdGroupFeedParams'] = TextAdGroupFeedParams_1

params = {}
params['AdGroups'] = [AdGroup]

service = 'dynamicfeedadtargets'
method = "add"
result = direct.requestAPI(service, method, params, login).json()
result

{'result': {'AddResults': [{'Id': 4965209059,
    'Warnings': [{'Code': 10165,
      'Message': 'Parameter will not be applied',
      'Details': 'Товарные дополнения на поиске больше не поддерживаются. Фиды не будут добавлены к группе объявлений.'}]}]}}

# SmartAdTargets - операций с фильтрами — условиями нацеливания для смарт-баннеров

Отбор целевой аудитории происходит по двум принципам.
* ретаргетинговый
* поведенческий

### add - Создадим РК с типом "SmartCampaign"

In [49]:
# https://yandex.ru/dev/direct/doc/ref-v5/campaigns/add.html#input__CampaignAddItem
# https://yandex.ru/dev/direct/doc/ref-v5/campaigns/add-text-campaign.html#add-text-campaign__TextCampaignAddItem

# Создание тела запроса
# Параметры типа РК (текстовая РК)
TypeParameters = {}
TypeParameters['BiddingStrategy'] = {}
TypeParameters['BiddingStrategy']['Search'] = {}
TypeParameters['BiddingStrategy']['Search']['BiddingStrategyType'] = 'AVERAGE_CPC_PER_FILTER'

TypeParameters['BiddingStrategy']['Search']['AverageCpcPerFilter'] = {}
TypeParameters['BiddingStrategy']['Search']['AverageCpcPerFilter']['FilterAverageCpc'] = 2000000


TypeParameters['BiddingStrategy']['Network'] = {}
TypeParameters['BiddingStrategy']['Network']['BiddingStrategyType'] = 'NETWORK_DEFAULT'


# Параметры РК
Campaign = {}
Campaign['Name'] = "Test campaigns 3 (SmartCampaign)"
Campaign['StartDate'] = "2026-01-01"

# Параметры типа РК (текстовая РК)
Campaign['SmartCampaign'] = TypeParameters

# Идентификатор счетчика Яндекс Метрики
Campaign['SmartCampaign']['CounterId'] = 65071099

params = {}
params['Campaigns'] = [Campaign]

service = 'campaigns'
method = "add"
result = direct.requestAPI(service, method, params, login).json()
result

{'result': {'AddResults': [{'Id': 76271726}]}}

### add - Создадим группу объявлений типа "SMART_AD_GROUP"

In [55]:
# https://yandex.ru/dev/direct/doc/ref-v5/adgroups/adgroups.html
# https://yandex.ru/dev/direct/doc/ref-v5/adgroups/add.html

# id РК
Campaigns_id = 76271726

# Массив идентификаторов регионов, для которых показы включены или выключены. Массив должен содержать хотя бы один элемент.
# Идентификатор 0 — показывать во всех регионах.
RegionIds = [0]

# Параметры AdGroup (группы объявлений)
AdGroup = {}
AdGroup['Name'] = "Тестовая группа - 03 (SMART_AD_GROUP)"
AdGroup['CampaignId'] = Campaigns_id
AdGroup['RegionIds'] = RegionIds

# Параметры группы с типом "SmartAdGroup"
AdGroup['SmartAdGroup'] = {}
# Идентификатор фида, на основе которого требуется сгенерировать смарт-баннеры
AdGroup['SmartAdGroup']['FeedId'] = 238509

params = {}
params['AdGroups'] = [AdGroup]

service = 'adgroups'
method = "add"
result = direct.requestAPI(service, method, params, login).json()
result

{'result': {'AddResults': [{'Id': 4968246761}]}}

### add - Создает фильтры — условия нацеливания для смарт-баннеров, назначает CPC, CPA или приоритет для создаваемых фильтров

**CPC, CPA** или приоритет используются в зависимости от того, какая стратегия показа выбрана в кампании:

* Если в кампании настроена стратегия **AVERAGE_ROI**, то используется параметр **StrategyPriority**: фильтры с более высоким приоритетом по возможности получают больший объем трафика на поиске или долю аудитории в сетях, а при недостатке бюджета отключаются в последнюю очередь.

* Если в кампании настроена стратегия **AVERAGE_CPA_PER_CAMPAIGN** или **AVERAGE_CPA_PER_FILTER**, то используется параметр **AverageCpa**.

* Если в кампании настроена стратегия **AVERAGE_CPC_PER_CAMPAIGN** или **AVERAGE_CPC_PER_FILTER**, то используется параметр **AverageCpc**.

Если указана ставка или приоритет, не соответствующие стратегии, они игнорируется.

Если стратегия предполагает назначение ставки, а ставка не указана, используется ставка из настроек стратегии. Если стратегия предполагает назначение приоритета, а приоритет не указан, возвращается ошибка.

Фильтры для смарт-баннеров можно добавить только в группу объявлений с типом **SMART_AD_GROUP**

In [57]:
# https://yandex.ru/dev/direct/doc/ref-v5/smartadtargets/smartadtargets.html
# https://yandex.ru/dev/direct/doc/ref-v5/smartadtargets/add.html


SmartAdTarget_1 = {}
SmartAdTarget_1['Name'] = "test_filtr_01"
SmartAdTarget_1['AdGroupId'] = 4968246761

# Целевая аудитория:
# INTERESTED_IN_SIMILAR_PRODUCTS — пользователи, которые интересовались похожими товарами в сети.
# VISITED_PRODUCT_PAGE — пользователи, которые смотрели товары на сайте рекламодателя.
# ALL_SEGMENTS — обе группы.
SmartAdTarget_1['Audience'] = 'ALL_SEGMENTS'


params = {}
params['SmartAdTargets'] = [SmartAdTarget_1]

service = 'smartadtargets'
method = "add"
result = direct.requestAPI(service, method, params, login).json()
result

{'result': {'AddResults': [{'Id': 1861265}]}}

### get - Возвращает параметры фильтров — условий нацеливания для смарт-баннеров

In [58]:
# https://yandex.ru/dev/direct/doc/ref-v5/smartadtargets/smartadtargets.html

params = {}
params["SelectionCriteria"] = {}
# Хотя бы один из параметров CampaignIds, AdGroupIds и Ids
# params["SelectionCriteria"]['CampaignIds'] = []
params["SelectionCriteria"]['AdGroupIds'] = [4968246761]
# params["SelectionCriteria"]['Ids'] = []

# Список выводимых полей - высокий уровень
params["FieldNames"] =[ "Id", "AdGroupId", "CampaignId", "Name", "AverageCpc", "AverageCpa", "StrategyPriority", "Conditions", "ConditionType", "State", "Audience", "AvailableItemsOnly" ]

service = 'smartadtargets'
method = "get"
result = direct.requestAPI(service, method, params, login).json()
result

{'result': {'SmartAdTargets': [{'Id': 1861265,
    'AdGroupId': 4968246761,
    'CampaignId': 76271726,
    'Name': 'test_filtr_01',
    'AverageCpc': None,
    'AverageCpa': None,
    'StrategyPriority': None,
    'State': 'OFF',
    'Audience': 'ALL_SEGMENTS',
    'Conditions': None,
    'ConditionType': 'ITEMS_ALL',
    'AvailableItemsOnly': 'NO'}]}}

### Еще методы

* **setBids** - Назначает CPC, CPA и приоритеты для фильтров. CPC, CPA или приоритет можно назначить для отдельного фильтра, для всех фильтров в группе объявлений или для всех фильтров в кампании
* **update**
* **suspend** - Останавливает показы по фильтрам
* **resume** - Возобновляет показы
* **delete**

# Changes - проверка наличия изменений

### checkCampaigns - наличии изменений в кампаниях клиента начиная с указанной даты

#### !Замечание: Рассматривает ВСЕ РК и выводит только РК

In [4]:
# https://yandex.ru/dev/direct/doc/ref-v5/changes/changes.html
# https://yandex.ru/dev/direct/doc/ref-v5/changes/checkCampaigns.html

# Где именно произошли изменения:
# SELF — в параметрах кампании;
# CHILDREN — в дочерних объектах: группах, объявлениях, фразах;
# STAT — корректировка статистики кампании (как правило, в связи с фильтрацией недобросовестных кликов).

# Создание тела запроса
params = {}
# Момент времени, начиная с которого проверяются изменения. Указывается в формате YYYY-MM-DDThh:mm:ssZ (согласно ISO 8601)
params["Timestamp"] = "2022-01-01T23:59:59Z"

service = 'changes'
method = "checkCampaigns"
result = direct.requestAPI(service, method, params, login).json()
result

{'result': {'Timestamp': '2022-08-03T05:46:01Z',
  'Campaigns': [{'ChangesIn': ['SELF', 'CHILDREN', 'STAT'],
    'CampaignId': 65628948},
   {'ChangesIn': ['SELF', 'CHILDREN', 'STAT'], 'CampaignId': 65628984},
   {'ChangesIn': ['SELF', 'CHILDREN'], 'CampaignId': 65628995},
   {'ChangesIn': ['SELF', 'CHILDREN', 'STAT'], 'CampaignId': 65629004},
   {'ChangesIn': ['SELF', 'CHILDREN'], 'CampaignId': 65629014},
   {'CampaignId': 66053066, 'ChangesIn': ['SELF', 'STAT']},
   {'CampaignId': 66054403, 'ChangesIn': ['SELF', 'CHILDREN', 'STAT']},
   {'CampaignId': 66089423, 'ChangesIn': ['SELF', 'CHILDREN']},
   {'ChangesIn': ['SELF', 'CHILDREN', 'STAT'], 'CampaignId': 66089435},
   {'ChangesIn': ['SELF', 'CHILDREN', 'STAT'], 'CampaignId': 66089444},
   {'ChangesIn': ['SELF', 'CHILDREN', 'STAT'], 'CampaignId': 66089460},
   {'ChangesIn': ['SELF', 'CHILDREN', 'STAT'], 'CampaignId': 66089478},
   {'ChangesIn': ['SELF', 'CHILDREN'], 'CampaignId': 66089484},
   {'ChangesIn': ['SELF', 'CHILDREN', 'STA

### check - наличии изменений в КОНКРЕТНОЙ РК (группе. объявлении)

#### !Замечание: Можно задавать один только 1 из ТРЕХ уровней: "РК - группа - объявление"

In [5]:
# https://yandex.ru/dev/direct/doc/ref-v5/changes/changes.html
# https://yandex.ru/dev/direct/doc/ref-v5/changes/check.html


# Создание тела запроса
params = {}
# Момент времени, начиная с которого проверяются изменения. Указывается в формате YYYY-MM-DDThh:mm:ssZ (согласно ISO 8601)
params["Timestamp"] = "2022-01-01T23:59:59Z"

# Либо CampaignIds, либо AdGroupIds, либо AdIds
params["CampaignIds"] = [66418007]
# params["AdGroupIds"] = []
# params["AdIds"] = []

# Список выводимых поей
params["FieldNames"] = ["CampaignIds", "AdGroupIds", "AdIds", "CampaignsStat"]

service = 'changes'
method = "check"
result = direct.requestAPI(service, method, params, login).json()
result

{'result': {'Timestamp': '2022-08-03T05:47:18Z',
  'Modified': {'AdGroupIds': [4717857118,
    4717857119,
    4717857120,
    4717857121,
    4717857122,
    4717857123,
    4717857124,
    4717857125,
    4717857126,
    4717857127,
    4717857128,
    4717857129,
    4717857130,
    4717857131,
    4717857132,
    4717857133,
    4717857134,
    4717857135,
    4717857136,
    4717857137,
    4717857138],
   'AdIds': [11256876425,
    11256876426,
    11256876427,
    11256876428,
    11256876429,
    11256876430,
    11256876431,
    11256876432,
    11256876433,
    11256876434,
    11256876435,
    11256876436,
    11256876437,
    11256876438,
    11256876439,
    11256876440,
    11256876441,
    11256876442,
    11256876443,
    11256876444,
    11256876445],
   'CampaignIds': [66418007],
   'CampaignsStat': [{'CampaignId': 66418007, 'BorderDate': '2022-02-03'}]}}}

# Dictionaries -  получения справочников (константы + параметры ЯД)

Сервис предназначен для получения справочных данных: регионов, часовых поясов, курсов валют, станций метрополитена, ограничений на значения параметров и др.

Что еще может вернуть:
* Версии операционных систем
* Социально-демографические характеристики и поведенческие признаки
* Сегменты по социально-демографическим характеристикам и поведенческим признакам для таргетинга по профилю пользователя.
* Сегменты по интересам пользователя для таргетинга по профилю пользователя

Ограничения на значения параметров:
* MaximumAdTextWordLength — максимальное количество символов в каждом слове в тексте объявления
* MaximumSitelinkTextLength — максимальное количество символов в тексте быстрой ссылки
* и т.д.

Для валютных параметров:
* BidIncrement — шаг торгов
* MaximumBid — максимальная ставка за клик (техническое ограничение)
* MinimumWeeklySpendLimit — минимальный недельный бюджет
* и тд.

### get - Возвращаем справочник "социально-демографических характеристик"

In [33]:
# https://yandex.ru/dev/direct/doc/ref-v5/dictionaries/dictionaries.html
# https://yandex.ru/dev/direct/doc/ref-v5/dictionaries/get.html

params = {}
params["DictionaryNames"] = ["AudienceCriteriaTypes"]

service = 'dictionaries'
method = "get"
result = direct.requestAPI(service, method, params, login).json()
# result

### get - Возвращаем справочник "социально-демографических характеристик"

In [36]:
# https://yandex.ru/dev/direct/doc/ref-v5/dictionaries/dictionaries.html
# https://yandex.ru/dev/direct/doc/ref-v5/dictionaries/get.html

params = {}
params["DictionaryNames"] = ["GeoRegions"]

service = 'dictionaries'
method = "get"
result = direct.requestAPI(service, method, params, login).json()
# result